# Final Project - Part 3 | Model

1. Identify and implement at least three different models.
    - Model choices must include at least one supervised and one unsupervised learning model.
    
2. The business objective can vary for supervised versus unsupervised model.

3. For each model prepare a report showing 
    - results obtained and hyperparameter values used
    - justification for dimensionality reduction (or not) including supporting data
    - resulting conclusions for each model regarding potential fit. 

# Approach

Part 3 of our final project will see implementations of three different approaches to solving the underlying questions on investment timing for TQQQ. First, before seeking the different methods on answering the question, I would like to restate the goal of our ML models. That is, to appropriately time the long, short, or hold of the TQQQ ETF. Although perfect market timing is impossible to achieve, according to the Efficient Market Hypothesis. Below will outline the expectations for each of the models and how I believe that these methods might accomplish more efficient entries and exits into the market.

## RNN

Through RNN, we are attempting to achieve predictions for 3, 5, 10, and 20 day future prices of the security. This approach will then allow us to use the investment function to decide how we will time our investment

# RNN Timeseries Market Data

# Classification Model - XGBoost?

# Segmentation Model - KMeans